<a href="https://colab.research.google.com/github/Mrsenjiii/Machine_learnning_algos/blob/main/Pipelines_and_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFpr

In [206]:
df = pd.read_csv('V5.csv')

In [207]:
df.head(2)

,Date,Year,Locality,Estimated Value,Sale Price,Property,Residential,num_rooms,num_bathrooms,carpet_area,property_tax_rate,Face
0,2009-01-02,2009,Waterbury,111440.0,185000.0,Single Family,Detached House,3,3,996.0,1.025953,South
1,2009-01-02,2009,Bridgeport,124670.0,150000.0,Two Family,Duplex,4,3,1241.0,1.025953,South


In [208]:
X = df.drop( ['Date' , 'Sale Price'] , axis=1)
y = df['Sale Price']
X.head(2)

,Year,Locality,Estimated Value,Property,Residential,num_rooms,num_bathrooms,carpet_area,property_tax_rate,Face
0,2009,Waterbury,111440.0,Single Family,Detached House,3,3,996.0,1.025953,South
1,2009,Bridgeport,124670.0,Two Family,Duplex,4,3,1241.0,1.025953,South


In [209]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=False)

In [210]:
X_train.isin( ['?' , np.nan]).sum()
# X_train.isin( ['?']).sum()
# X_train.isin( [np.nan]).sum()

Year                    0
Locality             1006
Estimated Value       977
Property             1464
Residential             0
num_rooms               0
num_bathrooms           0
carpet_area           957
property_tax_rate       0
Face                    0
dtype: int64

locality ---> Mode

Estimated Value ---> mean, median (Median)

carpet_area ----> median

property ----> Mode (most_frequent) "?"

In [211]:
si = SimpleImputer(strategy='most_frequent' , missing_values=np.nan)

#Columns Transformer for the data cleaning we will use it in Pipeline.

In [212]:
trf1 = [
    ( "lacalityImputer" ,  SimpleImputer(strategy='most_frequent' , missing_values=np.nan) , ['Locality']),
    ( "carpetarea_estimatedvalue" ,  SimpleImputer(strategy='median' , missing_values=np.nan ) , ['Estimated Value' ,'carpet_area']),
    ( "PropertyImputer" ,  SimpleImputer(strategy= 'most_frequent' , missing_values='?' ) ,['Property'])
]
ctf1 = ColumnTransformer(transformers = trf1 ,remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')
ctf1 ######### columns transformer 1 for data cleaning

ColumnTransformer(remainder='passthrough',
                  transformers=[('lacalityImputer',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['Locality']),
                                ('carpetarea_estimatedvalue',
                                 SimpleImputer(strategy='median'),
                                 ['Estimated Value', 'carpet_area']),
                                ('PropertyImputer',
                                 SimpleImputer(missing_values='?',
                                               strategy='most_frequent'),
                                 ['Property'])],
                  verbose_feature_names_out=False)

In [213]:
trfdf =  ctf1.fit_transform(X_train) ##########trasnformed data 1

In [214]:
trfdf.columns

Index(['Locality', 'Estimated Value', 'carpet_area', 'Property', 'Year',
       'Residential', 'num_rooms', 'num_bathrooms', 'property_tax_rate',
       'Face'],
      dtype='object')

In [215]:
trfdf.select_dtypes(exclude = 'number').columns

Index(['Locality', 'Property', 'Residential', 'Face'], dtype='object')

In [216]:
trfdf.select_dtypes(exclude = 'number').head(5)

,Locality,Property,Residential,Face
0,Waterbury,Single Family,Detached House,South
1,Bridgeport,Two Family,Duplex,South
2,Waterbury,Single Family,Detached House,South
3,Bridgeport,Single Family,Detached House,East
4,Bridgeport,Single Family,Detached House,East


#Columns Transformer for categorical encoding

In [217]:
trfdf['Locality'].value_counts() #need one hot encoding
# trfdf['Property'].value_counts() # ordinal encoding
# trfdf['Residential'].value_counts() # ordinal encoding
# trfdf['Face'].value_counts() # one hot encoding req

Bridgeport       2429
Waterbury        1225
Fairfield         923
West Hartford     890
Norwalk           872
Greenwich         842
Stamford          819
Name: Locality, dtype: int64

In [218]:
trf2 = [('ohe' , OneHotEncoder(sparse_output=False  , drop='first') , ['Locality','Face'] ) , # , drop='first' we could have done this also to prevent multicollinearity.
         ('ordinal' , OrdinalEncoder() , ['Property' , 'Residential'] )]
ctf2 = ColumnTransformer(transformers=trf2 , remainder='passthrough' , verbose_feature_names_out=False ).set_output(transform='pandas')
ctf2 # column transformer 2 for encoding

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['Locality', 'Face']),
                                ('ordinal', OrdinalEncoder(),
                                 ['Property', 'Residential'])],
                  verbose_feature_names_out=False)

In [219]:
trfdf2 = ctf2.fit_transform(trfdf) ################# transformed data trfd2

In [220]:
trfdf2

,Locality_Fairfield,Locality_Greenwich,Locality_Norwalk,Locality_Stamford,Locality_Waterbury,Locality_West Hartford,Face_North,Face_South,Face_West,Property,Residential,Estimated Value,carpet_area,Year,num_rooms,num_bathrooms,property_tax_rate
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,111440.0,996.0,2009,3,3,1.025953
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,124670.0,1241.0,2009,4,3,1.025953
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,55720.0,910.0,2009,3,2,1.025953
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4775276.0,971.0,2009,3,1,1.025953
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,112351.0,1092.0,2009,3,2,1.025953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,194370.0,918.0,2020,3,3,1.025440
7996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,160910.0,1021.0,2020,4,3,1.025440
7997,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,354480.0,1084.0,2020,3,2,1.025440
7998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,229530.0,946.0,2020,3,3,1.025440


In [221]:
X_train.select_dtypes(exclude="number").columns # ignore these features from train because already in 0 to 1 or in less scale on rest apply minmaxscaler

Index(['Locality', 'Property', 'Residential', 'Face'], dtype='object')

In [222]:

stp = [
    ('Data_cleaning',ctf1),
    ('Encoding_Ordinal_Nominal' , ctf2),
    ('Scaling_MinMax' , StandardScaler().set_output(transform='pandas')),
    ("PCA Feature Reduction" , PCA(n_components=15)),
    ('SGD LInear Model' , SGDRegressor())
]

pipe = Pipeline(steps=stp).set_output(transform='pandas')

In [223]:
pipe # wow what a complex pipeline.

Pipeline(steps=[('Data_cleaning',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('lacalityImputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Locality']),
                                                 ('carpetarea_estimatedvalue',
                                                  SimpleImputer(strategy='median'),
                                                  ['Estimated Value',
                                                   'carpet_area']),
                                                 ('PropertyImputer',
                                                  SimpleImputer(missing_values='?',
                                                                strategy='most_frequent'),
                                                  ['Property'])]...
                ('Encoding_Ordinal_Nominal',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Locality', 'Face']),
                                                 ('ordinal', OrdinalEncoder(),
                                                  ['Property', 'Residential'])],
                                   verbose_feature_names_out=False)),
                ('Scaling_MinMax', StandardScaler()),
                ('PCA Feature Reduction', PCA(n_components=15)),
                ('SGD LInear Model', SGDRegressor())])

In [224]:
pipe.fit(X_train , y_train)

Pipeline(steps=[('Data_cleaning',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('lacalityImputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Locality']),
                                                 ('carpetarea_estimatedvalue',
                                                  SimpleImputer(strategy='median'),
                                                  ['Estimated Value',
                                                   'carpet_area']),
                                                 ('PropertyImputer',
                                                  SimpleImputer(missing_values='?',
                                                                strategy='most_frequent'),
                                                  ['Property'])]...
                ('Encoding_Ordinal_Nominal',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Locality', 'Face']),
                                                 ('ordinal', OrdinalEncoder(),
                                                  ['Property', 'Residential'])],
                                   verbose_feature_names_out=False)),
                ('Scaling_MinMax', StandardScaler()),
                ('PCA Feature Reduction', PCA(n_components=15)),
                ('SGD LInear Model', SGDRegressor())])

In [225]:
pipe.score(X_train,y_train)

0.7030702980149578

In [226]:
pipe.score(X_test,y_test)

0.7827701795452677